In [1]:
import numpy as np

class Ciphertext:
    def __init__(self, data : np.array, maxLevel : int, level : int =None):
        self.data = data  # 입력된 벡터를 저장
        self.maxLevel = maxLevel
        self.level= level if level is not None else maxLevel
    def get_data(self):
        return self.data

    def get_level(self):
        return self.level

    def get_maxLevel(self):
        return self.maxLevel   


# 1. ScalarMult: Ciphertext 객체와 벡터의 성분별 곱셈 결과를 반환
def ScalarMult(ciphertext, vector):
    maxLevel = ciphertext.get_maxLevel()
    level = ciphertext.get_level() - 1
    if level < 0:
        raise ValueError("error, Mult is not available if the level is below 0.")
    result = np.multiply(ciphertext.get_data(), vector)
    return Ciphertext(result, maxLevel,level)

# 2. NonscalarMult: 두 Ciphertext 객체의 성분별 곱셈 결과를 반환
def NonscalarMult(ciphertext1, ciphertext2):
    maxLevel = ciphertext1.get_maxLevel()
    level = min(ciphertext1.get_level(),ciphertext2.get_level()) - 1
    if level < 0:
        raise ValueError("error, Mult is not available if the level is below 0.")
    result = np.multiply(ciphertext1.get_data(), ciphertext2.get_data())
    return Ciphertext(result, maxLevel,level)

# 3. ScalarAdd: Ciphertext 객체와 벡터의 성분별 덧셈 결과를 반환
def ScalarAdd(ciphertext, vector):
    maxLevel = ciphertext.get_maxLevel()
    level = ciphertext.get_level()
    result = np.add(ciphertext.get_data(), vector)
    return Ciphertext(result,maxLevel,level)

# 4. NonscalarAdd: 두 Ciphertext 객체의 성분별 덧셈 결과를 반환
def NonscalarAdd(ciphertext1, ciphertext2):
    maxLevel = ciphertext1.get_maxLevel()
    level = min(ciphertext1.get_level(),ciphertext2.get_level())
    result = np.add(ciphertext1.get_data(), ciphertext2.get_data())
    return Ciphertext(result,maxLevel,level)

# 5. Rotation: Ciphertext 객체의 벡터를 왼쪽으로 r만큼 cyclic shift한 결과를 반환
def Rotation(ciphertext, r):
    maxLevel = ciphertext.get_maxLevel()
    level = ciphertext.get_level()
    data = ciphertext.get_data()
    n = len(data)
    r = r % n  # r이 벡터 길이를 초과하는 경우를 대비해 모듈로 연산
    result = np.concatenate((data[r:], data[:r]))
    return Ciphertext(result,maxLevel,level)
# 6. Bootstrap
def Bootstrap(ciphertext):
    #실행 시간이 매우 깁니다...
    print("매우 오랜 시간이 지난 뒤...")
    ciphertext.level = ciphertext.maxLevel

# # 사용 예시
# ct1 = Ciphertext([1, 2, 3, 4, 5])
# ct2 = Ciphertext([6, 7, 8, 9, 10])
# vector = [2, 3, 4, 5, 6]

# # ScalarMult 예시
# result_scalar_mult = ScalarMult(ct1, vector)
# print("ScalarMult 결과:", result_scalar_mult.get_data())

# # NonscalarMult 예시
# result_nonscalar_mult = NonscalarMult(ct1, ct2)
# print("NonscalarMult 결과:", result_nonscalar_mult.get_data())

# # ScalarAdd 예시
# result_scalar_add = ScalarAdd(ct1, vector)
# print("ScalarAdd 결과:", result_scalar_add.get_data())

# # NonscalarAdd 예시
# result_nonscalar_add = NonscalarAdd(ct1, ct2)
# print("NonscalarAdd 결과:", result_nonscalar_add.get_data())

# # Rotation 예시
# result_rotation = Rotation(ct1, 2)
# print("Rotation 결과:", result_rotation.get_data())

CKKS의 연산은 기본적으로 어떤 연산이든 계속해서 사용가능하지만, 곱셈만큼은 예외다.   
모든 암호문에는 level이 따로 저장되며 모든 논스칼라 연산은 상대적으로 level이 낮은 쪽으로 맞춰진다.   
그리고, 스칼라/논스칼라 여부 상관 없이 곱셈은 한번 사용할때마다 level이 하나씩 줄어든다.

In [2]:
ct1 = Ciphertext([1, 2, 3, 4, 5], 5)
ct2 = Ciphertext([6, 7, 8, 9, 10], 3)
vector = [2, 3, 4, 5, 6]

# 논스칼라 덧셈 (레벨이 서로 다른)
print("non-scalar add")
after_NonscalarAdd = NonscalarAdd(ct1,ct2)
print(f"data : {ct1.get_data()}, {ct2.get_data()} --> {after_NonscalarAdd.get_data()}")
print(f"before level : {ct1.get_level()}, {ct2.get_level()} --> after level = {after_NonscalarAdd.get_level()}\n")


# 스칼라 곱셈
print("scalar mult")
after_ScalarMult = ScalarMult(ct1,vector)
print(f"data : {ct1.get_data()} --> {after_ScalarMult.get_data()}")
print(f"before level = {ct1.get_level()} --> after level = {after_ScalarMult.get_level()}\n")


# 논스칼라 곱셈
print("non-scalar mult")
after_NonscalarcalarMult = NonscalarMult(ct1,ct2)
print(f"data : {ct1.get_data()}, {ct2.get_data()} --> {after_NonscalarcalarMult.get_data()}")
print(f"before level : {ct1.get_level()}, {ct2.get_level()} --> after level = {after_NonscalarcalarMult.get_level()}\n")

non-scalar add
data : [1, 2, 3, 4, 5], [6, 7, 8, 9, 10] --> [ 7  9 11 13 15]
before level : 5, 3 --> after level = 3

scalar mult
data : [1, 2, 3, 4, 5] --> [ 2  6 12 20 30]
before level = 5 --> after level = 4

non-scalar mult
data : [1, 2, 3, 4, 5], [6, 7, 8, 9, 10] --> [ 6 14 24 36 50]
before level : 5, 3 --> after level = 2



level이 0이 되면 더 이상 곱셈을 수행할 수 없다.   
다시 level을 초기 값으로 회복시키기 위해서는 Bootstrapping이라는 작업을 해야하며, 이는 매우매우 시간이 오래 걸리는 작업이다.   
그렇기에 CKKS내에서의 딥러닝 수행의 최우선 과제는 Bootstrapping의 시간효율을 올리거나, 곱셈의 횟수(level의 소모)를 줄이는 것이다.   
(참고로, 본 과제에서는 Bootstrapping을 사용하지 않는다.)

In [3]:
# ct1의 maxlevel=1, level = 1
ct1 = Ciphertext([1, 2, 3, 4, 5], 1)
vector = [2, 3, 4, 5, 6]

# after_ScalarMult의 level = 0
after_ScalarMult = ScalarMult(ct1,vector)
print(f"data : {after_ScalarMult.get_data()}")
print(f"before level = {ct1.get_level()} --> after level = {after_ScalarMult.get_level()}\n")
try : 
    #level 0에서 곱셈 수행 -> 에러
    after_after_ScalarMult = ScalarMult(after_ScalarMult,vector)
except ValueError as e:
    print(e)

print("\n")

## 부트스트래핑, 레벨을 초기 레벨로 회복(이 경우, 1)
print("부트스트래핑 실행")
Bootstrap(after_ScalarMult)
after_after_ScalarMult = ScalarMult(after_ScalarMult,vector)
print(f"data : {after_after_ScalarMult.get_data()}")
print(f"before level = {ct1.get_level()}\nafter level = {after_after_ScalarMult.get_level()}")

data : [ 2  6 12 20 30]
before level = 1 --> after level = 0

error, Mult is not available if the level is below 0.


부트스트래핑 실행
매우 오랜 시간이 지난 뒤...
data : [  4  18  48 100 180]
before level = 1
after level = 0


### 문제1.

$x^8$은 일반적으로 생각해보면, level을 7만큼 소모해야한다. ($x \times x$ 한번, $x \times x \times x$ 두번, $x \times x \times x \times x$ 세번 ...)   
하지만 level을 단 3만 소모해서 구현할수 있다. 이를 구현하시오   


* * *


Tip : $x \times x \times x$의 level은 2를 소모하였고, x은 level을 소모하지 않은 상태이다.   
이 둘을 곱하면 레벨이 더 낮은 쪽을(소모값이 더 큰쪽을) 기준으로 맞추어지므로 소모값은 $(이전소모값)+1=2+1=3$ 이 된다.   
하지만, $x \times x$는 level을 1만 소모하였으니 $(x \times x) \times (x \times x)$의 level 소모량은 $1+1=2$가 된다.

***

In [4]:
ct = Ciphertext([0,0.1, 0.2, 0.3, 0.4, 0.5], 3)

## 아래 공간을 채워 넣어주세요
pass
x = ct
x2 = NonscalarMult(x,x)
x4 = NonscalarMult(x2,x2)
x8 = NonscalarMult(x4,x4)

result = x8
##

print(f"\ndata : {result.get_data()}")
print(f"before level = {ct.get_level()}, after level = {result.get_level()}")


data : [0.00000e+00 1.00000e-08 2.56000e-06 6.56100e-05 6.55360e-04 3.90625e-03]
before level = 3, after level = 0


### 문제2.

아래의 방정식을 과제의 1번 문제와 연계하여 최소한의 레벨로 구현하여 풀어보시오   
$5x^7 - x^4 - 2x^3 + 7x^2 + 2x + 2$

In [5]:
ct = Ciphertext([0,0.1, 0.2, 0.3, 0.4, 0.5], 4)

## 아래 공간을 채워 넣어주세요
pass
x = ct
x2 = NonscalarMult(x,x)
x3 = NonscalarMult(x2,x)
x4 = NonscalarMult(x2,x2)
x5 = NonscalarMult(x4,x)
x6 = NonscalarMult(x4,x2)
x7 = NonscalarMult(x4,x3)

temp = ScalarMult(x7,5)
temp = NonscalarAdd(temp,ScalarMult(x4,-1))
temp = NonscalarAdd(temp,ScalarMult(x3,-2))
temp = NonscalarAdd(temp,ScalarMult(x2,7))
temp = NonscalarAdd(temp,ScalarMult(x,2))
temp = ScalarAdd(temp,2)
result = temp
##

print(f"\ndata : {result.get_data()}")
print(f"before level = {ct.get_level()}, after level = {result.get_level()}")


data : [2.        2.2679005 2.662464  3.1689935 3.774592  4.4765625]
before level = 4, after level = 0


### 문제3. (고난이도)

CKKS는 다항함수만 동형적으로 연산이 가능하지만 딥러닝에서 다항함수만 사용할수는 없는 노릇이다.   
특히 activation 함수는 비 다항함수만을 사용하기에 더욱 문제가 된다.

하지만 activation 함수 중 가장 많이 쓰이는 ReLU 함수는 다항함수로 잘 근사하는 방법이 많이 연구되었고,   
주어진 parameter $a$에 대해 $(error) <= 2^{-a}$ 내에서 ReLU를(정확히는 sign함수를) 근사하는 방법을 일반화 하여 구현하는 데에 성공하였다.

가령 이 논문에서 제시한 $a=11$기준, 적합한 근사 다항식의 차수는 7차, 7차, 27차의 합성함수이며 이는 다음과 같다.

![Alt text](data/img.png)

$p_α(x) = p_{α,3} ◦ p_{α,2} ◦ p_{α,1}(x)$이며 결과적으로 $p_α(x)$는 sign함수를 근사한 함수이다.   
$r_α(x) = \frac {x+xp_α(x)} {2}$이며 결과적으로 r_α(x)은 ReLU함수를 근사한 함수이다.

근사한 다항식의 계수를 불러오는 코드가 아래 코드블럭에서 미리 구현되어있다.
불러오는 계수는 $alpha=8$일 때 sign함수의 근사 다항식의 계수값이이며 7차, 15차의 합성함수 형태이다.
불러온 계수로 sign함수를 근사하고, 이를 통해 ReLU를 구현하시오   
(본 근사 방식은 $[-1,1]$ 내에서만 동작한다. 그러므로 만일 이 범위 바깥의 숫자를 넣고 싶다면 sign함수의 입력 값을 큰 값(B)으로 나누고 입력하면 된다.)

In [ ]:
def parse_sign_coeffs(file_path):
    def parse_scientific_notation(s):
        # 공백으로 분리된 문자열을 하나로 합치고 'e'로 변환
        s = s.replace(" ", "").replace("e-", "e-").replace("e+", "e+")
        return float(s)

    coeffs = {}
    current_alpha = None
    current_subgroup = None
    
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()
            if not line:
                continue
                
            # alpha 그룹 확인
            if line.startswith('alpha'):
                current_alpha = line.strip()
                coeffs[current_alpha] = {}
            # 하위 그룹 번호 확인
            elif line.isdigit():
                current_subgroup = int(line)
                coeffs[current_alpha][current_subgroup] = []
            # 계수값 처리
            elif current_alpha and current_subgroup is not None:
                try:
                    index, *value_parts = line.split()
                    value = parse_scientific_notation("".join(value_parts))
                    coeffs[current_alpha][current_subgroup].append((int(index), value))
                except ValueError:
                    continue
    
    return coeffs



coeffs = parse_sign_coeffs("data/sign_coeffs.txt")["alpha8"]

## 파싱 결과를 확인하기 위한 예시 코드입니다. 구조를 파악하고 싶다면 주석을 풀고 실행해보세요
# for key, value in coeffs.items():
#     print(f"{key}번째 다항식의 ")
#     for i in value:
#         print(f"{i[0]}차 항의 계수 : {i[1]}")
#     print()


ct = Ciphertext([0.1, 0.2, 0.3, 0.4, 0.5, -0.1,-0.2,-0.3,-0.4,-0.5], 20)


## 위에서 불러온 2개의 다항식을 이용해서 암호문 상에서 Sign함수를 구현하고, 이를 토대로 ReLU를 구현하시오, 그리고 마지막으로 ct를 입력하여 암호문 상에서 ReLU결과가 나오는 코드를 작성하시오
## 7차와 15차의 합성함수입니다, 7차의 결과물을 15차에 대입해서 집어넣는 식으로 구현하세요
## 아래 공간을 채워 넣으세요
pass
x = ct
x2 = NonscalarMult(x,x)
x3 = NonscalarMult(x2,x)
x4 = NonscalarMult(x2,x2)
x5 = NonscalarMult(x4,x)
x6 = NonscalarMult(x4,x2)
x7 = NonscalarMult(x4,x3)

temp = ScalarMult(x7,coeffs[1][7][1])
temp = NonscalarAdd(temp,ScalarMult(x6,coeffs[1][6][1]))
temp = NonscalarAdd(temp,ScalarMult(x5,coeffs[1][5][1]))
temp = NonscalarAdd(temp,ScalarMult(x4,coeffs[1][4][1]))
temp = NonscalarAdd(temp,ScalarMult(x3,coeffs[1][3][1]))
temp = NonscalarAdd(temp,ScalarMult(x2,coeffs[1][2][1]))
temp = NonscalarAdd(temp,ScalarMult(x,coeffs[1][1][1]))
temp = ScalarAdd(temp,coeffs[1][0][1])
result = temp


x = result
x2 = NonscalarMult(x,x)
x3 = NonscalarMult(x2,x)
x4 = NonscalarMult(x2,x2)
x5 = NonscalarMult(x4,x)
x6 = NonscalarMult(x4,x2)
x7 = NonscalarMult(x4,x3)
x8 = NonscalarMult(x4,x4)
x9 = NonscalarMult(x8,x)
x10 = NonscalarMult(x8,x2)
x11 = NonscalarMult(x8,x3)
x12 = NonscalarMult(x8,x4)
x13 = NonscalarMult(x8,x5)
x14 = NonscalarMult(x8,x6)
x15 = NonscalarMult(x8,x7)


temp = ScalarMult(x15,coeffs[2][15][1])
temp = NonscalarAdd(temp,ScalarMult(x14,coeffs[2][14][1]))
temp = NonscalarAdd(temp,ScalarMult(x13,coeffs[2][13][1]))
temp = NonscalarAdd(temp,ScalarMult(x12,coeffs[2][12][1]))
temp = NonscalarAdd(temp,ScalarMult(x11,coeffs[2][11][1]))
temp = NonscalarAdd(temp,ScalarMult(x10,coeffs[2][10][1]))
temp = NonscalarAdd(temp,ScalarMult(x9,coeffs[2][9][1]))
temp = NonscalarAdd(temp,ScalarMult(x8,coeffs[2][8][1]))
temp = NonscalarAdd(temp,ScalarMult(x7,coeffs[2][7][1]))
temp = NonscalarAdd(temp,ScalarMult(x6,coeffs[2][6][1]))
temp = NonscalarAdd(temp,ScalarMult(x5,coeffs[2][5][1]))
temp = NonscalarAdd(temp,ScalarMult(x4,coeffs[2][4][1]))
temp = NonscalarAdd(temp,ScalarMult(x3,coeffs[2][3][1]))
temp = NonscalarAdd(temp,ScalarMult(x2,coeffs[2][2][1]))
temp = NonscalarAdd(temp,ScalarMult(x,coeffs[2][1][1]))
temp = ScalarAdd(temp,coeffs[2][0][1])


result = temp
result = ScalarMult(NonscalarAdd(NonscalarMult(ct,result),ct),0.5)
##
print(f"\ndata : {result.get_data()}")
print(f"before level = {ct.get_level()}, after level = {result.get_level()}")



data : [ 1.00203541e-01  1.99714102e-01  2.99527775e-01  4.01341048e-01
  5.01266639e-01  2.03540619e-04 -2.85897859e-04 -4.72225130e-04
  1.34104784e-03  1.26663943e-03]
before level = 20, after level = 9


### 문제5. (고난이도)

CIFAR-10 이미지 분류에 탁월한 성능을 보였던 [ResNet-20](https://www.researchgate.net/figure/ResNet-20-architecture_fig3_351046093, "resnet-20")이라는 모델이 있다.   
많은 연구를 통해 CKKS내에서 ReNet-20을 구현을 성공적으로 하였다. 이는 CNN, BN, Pool, ReLU, FC-Layer등의 레이어를 구현해야한다.   
본 과제의 4번까지 잘 구현했다는 가정하에 우리는 ReLU를 성공적으로 구현했을것이다.   
그렇다면, ResNet-20의 ReLU를 교체한다면 제대로 동작하는지 확인해보는것이 본 문제이다.

아래 코드블럭에 이미 학습을 마친 ResNet-20모델을 불러왔다. 이 모델의 ReLU를 문제 4에서 구현한 ReLU로 바꾸어서 실행해 보자 그리고, 정확도를 비교하시오

***

Tip1: 이 모델은 이미 학습이 되어있는 모델이며, ReLU를 대체하지 않았을 때의 정확도는 91.46%이다   
Tip2: 본 과제에서는 alpha8, alpha11, alpha14일 때의 근사 다항식의 계수가 주어져있다. 각각 바꾸어 가면서 정확도를 비교해보세요   
Tip3: 문제 4번에서도 언급을 했듯이, 입력 값을 $[-1,1]$구간에 맞추어야한다.   
이때의 스케일링하는 값(B)은 각 alpha에 따라 적정하게 바꾸어 가면서 정확도를 비교하자.   
(B값의 적정값은 alpha8일때는 10~15, alpha11일때는 30~35, alpha14 일때는 285~290 사이에 있다.)

***

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

#ResNet-20의 BasicBlock 정의입니다, 되도록이면 변경하지 말아주세요
class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, relu, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )
        
        self.relu= relu
    
    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = self.relu(out)
        return out

# ResNet-20 정의입니다, 되도록이면, 변경하지 말아주세요
class ResNet20(nn.Module):
    def __init__(self, relu, num_classes=10):
        super(ResNet20, self).__init__()
        self.in_channels = 16
        
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.relu= relu

        self.layer1 = self._make_layer(16, 3, stride=1)
        self.layer2 = self._make_layer(32, 3, stride=2)
        self.layer3 = self._make_layer(64, 3, stride=2)
        
        self.linear = nn.Linear(64, num_classes)
        
    
    def _make_layer(self, out_channels, blocks, stride):
        strides = [stride] + [1] * (blocks - 1)
        layers = []
        for stride in strides:
            layers.append(BasicBlock(self.in_channels, out_channels, self.relu, stride))
            self.in_channels = out_channels
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.avg_pool2d(out, 8)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

# 문제 4에서 구현한 ReLU를 구현하세요
class CustomReLU(nn.Module):
    def __init__(self, coeffs_path = "data/sign_coeffs.txt", alpha='alpha11',B = 31):
        super(CustomReLU,self).__init__()
        ## 아래 공간을 채워 넣으세요
        pass
        parsed_coeffs = parse_sign_coeffs(coeffs_path)
        if alpha not in parsed_coeffs :
            raise ValueError(f"Invalid alpha ({alpha})")
        self.alpha = alpha
        self.parsed_coeffs = parsed_coeffs[alpha]
        self.B = B
        ##

    def forward(self,x):
        ## 아래 공간을 채워 넣으세요
        pass
        input = x / self.B
        for key in sorted(self.parsed_coeffs.keys()):
            coeffs = self.parsed_coeffs[key]
            result = 0
            for index, coeff in coeffs:
                result += coeff * (input ** index)
            input = result
        sign = input
        out=(sign * x + x)/2
        return out
        ##


# 위에서 구현한 ReLU를 불러오는 코드입니다. alpha는 alpha8, alpha11, alpha14 중 하나를 입력하세요
# B는 입력 값을 [-1,1]구간에 맞추기 위해 스케일링하는 값입니다. alpha값에 따라 적정 값이 다릅니다. 한번 직접 해보세요
relu = CustomReLU(alpha='alpha8',B=12.3)

## 실행 관련 코드들입니다. 되도록이면 수정하지 말아주세요
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ResNet20(relu,num_classes=10).to(device)
model.load_state_dict(torch.load("resnet20_cifar10.pth", map_location=device))
model.eval()


transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # 정규화
])
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False, num_workers=2)

# 추론
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"alpha : {relu.alpha}, B : {relu.B}")
print(f"Test Accuracy: {100 * correct / total:.2f}%")

C:\Users\BRAIN_3\AppData\Local\Temp\ipykernel_19036\2727737091.py:104: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("resnet20_cifar10.pth",

100%|██████████| 170M/170M [00:15<00:00, 11.0MB/s] 


Extracting ./data\cifar-10-python.tar.gz to ./data
alpha : alpha11, B : 31.0
Test Accuracy: 90.83%
